In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
dataset=pd.read_csv('/kaggle/input/turkish-airlines-daily-stock-prices-since-2013/cleanThy.csv')
dataset=dataset[::-1] #Inverting Dataset
dataset.reset_index(inplace=True)
dataset.drop(['index'],axis=1,inplace=True)
dataset.head(5)

In [ ]:
dataset.info()

# Graphs

In [ ]:
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
import plotly.graph_objects as go
trace=go.Scatter(x=dataset.index, y=dataset.iloc[:,1],
                    mode='lines',
                    name='Last Price',
                    marker = dict(color = 'red'))
layout = dict(title = 'Turkish Airlines Last Price 2013-2019',
              xaxis= dict(title= 'Time'),
              yaxis=dict(title='Price')
             )
fig = dict(data = trace, layout = layout)

iplot(fig)

In [ ]:
from plotly.offline import iplot
import plotly.graph_objects as go
trace=go.Scatter(x=dataset.index, y=dataset.iloc[:,2],
                    mode='lines',
                    name='Lowest Price',
                    marker = dict(color = 'blue'))
layout = dict(title = 'Turkish Airlines Lowest Price 2013-2019',
              xaxis= dict(title= 'Time'),
              yaxis=dict(title='Price')
             )
fig = dict(data = trace, layout = layout)

iplot(fig)

In [ ]:
from plotly.offline import iplot
import plotly.graph_objects as go
trace=go.Scatter(x=dataset.index, y=dataset.iloc[:,3],
                    mode='lines',
                    name='Highest Price',
                    marker = dict(color = 'grey'))
layout = dict(title = 'Turkish Airlines Highest Price 2013-2019',
              xaxis= dict(title= 'Time'),
              yaxis=dict(title='Price')
             )
fig = dict(data = trace, layout = layout)

iplot(fig)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dataset.index, y=dataset.iloc[:,1],
                    mode='lines',
                    name='Last Price'))
fig.add_trace(go.Scatter(x=dataset.index, y=dataset.iloc[:,2],
                    mode='lines',
                    name='Lowest Price'))
fig.add_trace(go.Scatter(x=dataset.index, y=dataset.iloc[:,3],
                    mode='lines',
                    name='Highest Price'))
fig.update_layout(title='Turkish Airlines Highest Price-Lower Price-Last Price',
                  xaxis_title='Time',
                  yaxis_title='Price')

iplot(fig)

# Splitting Dataset

## Timestep=80 days, Test Set=100 days

In [ ]:
training_set=dataset.iloc[0:1402,1:2].values
test_set = dataset.iloc[1322:,1:2].values #Last 100+80 days because of timestep

The model will try to predict the last 100 days of Last Stock Price

In [ ]:
#Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=[0,1])
training_set_scaled=sc.fit_transform(training_set)

In [ ]:
#Creating a data structure with 60 timesteps and 1 Output
X_train=[]
y_train=[]

for i in range(80,len(training_set_scaled)): 
    X_train.append(training_set_scaled[i-80:i,0])
    y_train.append(training_set_scaled[i,0]) 

X_train,y_train=np.array(X_train),np.array(y_train)

#Reshaping
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1)) #3 Dimension

# Building RNN

In [ ]:
## Step 2 Building RNN,

#Importing the Keras Library
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

#Initialising the RNN
regressor = Sequential()

#Adding LSTM layers Dropout regularisation
regressor.add(LSTM(units=55,return_sequences=True,input_shape = (X_train.shape[1], 1))) 

regressor.add(Dropout(rate=0.15)) #Dropout Regularization

regressor.add(LSTM(units=55,return_sequences=True))
regressor.add(Dropout(rate=0.15))

regressor.add(LSTM(units=55,return_sequences=True))
regressor.add(Dropout(rate=0.15))

regressor.add(LSTM(units=55))
regressor.add(Dropout(rate=0.15))

#Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.summary()

In [ ]:
# Fitting the RNN
regressor.fit(X_train,y_train, epochs = 64,batch_size=20)

# Testing and Results

Let's test our model with the last 100 days of Last Stock Price

In [ ]:
inputs=sc.transform(test_set)
X_test=list()
for i in range(80,180):
    X_test.append(inputs[i-80:i, 0])



X_test=np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

predicted_stock_price=regressor.predict(X_test)
predicted_stock_price=sc.inverse_transform(predicted_stock_price)

In [ ]:
real_stock_price=dataset.iloc[1402:,1:2]
real_stock_price.reset_index(inplace=True)
real_stock_price.drop(['index'],axis=1,inplace=True)
real_stock_price=real_stock_price.values

total_data=np.concatenate((real_stock_price, predicted_stock_price), axis=1)
total_data=pd.DataFrame(data=total_data,columns=['Real Stock Price','Predicted Stock Price'])
total_data

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=total_data.index, y=total_data['Real Stock Price'],
                    mode='lines',
                    name='Real Turkish Airlines Stock Price(Last)'))
fig.add_trace(go.Scatter(x=total_data.index, y=total_data['Predicted Stock Price'],
                    mode='lines',
                    name='Predicted Turkish Airlines Stock Price(Last)'))
fig.update_layout(title='Predicted Price vs Real Price',
                  xaxis_title='Time',
                  yaxis_title='Price')

iplot(fig)

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,median_absolute_error,r2_score

print('MAE=%0.2f'%mean_absolute_error(real_stock_price,predicted_stock_price))
print('MSE=%0.2f'%mean_squared_error(real_stock_price,predicted_stock_price))
print('MedAE=%0.2f'%median_absolute_error(real_stock_price,predicted_stock_price))
print('RMSE=%0.2f'%np.sqrt(mean_squared_error(real_stock_price,predicted_stock_price)))
print('R2 Score=%0.2f'%r2_score(real_stock_price,predicted_stock_price))